Problem 1. 
===
Create a computational graph for the following expression:
$$(x*y + z+1/x)* w = f$$
Calculate the forward values of all the nodes and function f starting with $x = -1, y = 2, z = 4, w = 5$. Subsequently, determine backward values, and finally the derivatives of  f with respect with x, y, z and w. Please, present your results as a simple graph. You can draw you graph by any means you find convenient, including by hand. Please place forward values above the lines representing propagation of values and backpropagation values (derivatives) below the lines.
(25%)

Solution:
---
* Black numbers are calculations for f starting with the given x, y, z, w values
* Red numbers are back-propagated derivative values
* NOTE: as there are two calculation paths from x, we need to add up the two back propagated derivative values to x
![hw3_p1.png](hw3_p1.png)

Problem 2. 
===
Create a computational graph for the following expression:
$$f =\frac{x+ \sigma(y)}{\sigma(z)+(x+y)^2}$$
Where$$\sigma(q) = \frac{1}{1+ e^{-q}}$$
Calculate forward computational values of all nodes and the derivatives of function f with respect to x,y and z. Please, present your results as a simple graph. Please place forward values above the lines representing propagation of values and backpropagation values (derivatives) below the lines. Perform your calculations manually. 
As values for x,y,z use (2, -4, 3)
(25%)

Solution:
---
* Representing Sigmoid function as a single node
* Black numbers are calculations for f starting with given x, y, z values
* Red numbers are back-propagated derivative values with precision of 4
![hw3_p2.png](hw3_p2.png)

Problem 3. 
===
Please perform calculations in problem 2 using TensorFlow. As you are moving forward, please calculate and store (cash) values of various derivatives you will need for backward calculations. 
(25%)

In [16]:
import tensorflow as tf

def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))

def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = tf.placeholder(tf.float32)

x_sigY = x + sigma(y)

x_y_2 = (x + y)**2

sigZ_x_y_2 = sigma(z) + x_y_2

f = tf.div(x_sigY, sigZ_x_y_2)


# Back propagation

# Upper branch of x
d_x_1 = tf.div(tf.constant(1, dtype=tf.float32), sigZ_x_y_2)

# Upper branch of y
d_y_1 = tf.multiply(sigmaprime(y), d_x_1)

d_y_z = tf.div(tf.negative(x_sigY), sigZ_x_y_2**2)
d_x_y_2 = 2*(x+y)

d_y_2 = d_x_y_2 * d_y_z
d_x_2 = d_y_2

d_x = d_x_1 + d_x_2
d_y = d_y_1 + d_y_2
d_z = tf.multiply(sigmaprime(z), d_y_z)


with tf.Session() as sess:
    result = sess.run([f, d_x, d_y, d_z], feed_dict={x: 2, y: -4, z: 3})
    print("Forward Calculated  f  =  {}".format(result[0]))
    print("Back propagation   dx  =  {}".format(result[1]))
    print("Back propagation   dy  =  {}".format(result[2]))
    print("Back propagation   dz  =  {}".format(result[3]))


Forward Calculated  f  =  0.40746209025382996
Back propagation   dx  =  0.5310063362121582
Back propagation   dy  =  0.33265751600265503
Back propagation   dz  =  -0.003716809442266822


Problem 4. 
===
Please examine attached Python code for classification of MNIST hand written digits dataset. Please try to find “optimal” values of two hyper parameters: learning_rate and batch_size. Optimal is a vague term. You would like to achieve the best accuracy in the shortest possible time. Please do not sweat it out. 
(25%)